In [ ]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import load_data
from column_checks import get_dtypes
from filepaths import *
import output_data
import emissions
import validation




# Load eGRID Emission Factors

Need to load CHP adjustment column to create "_for_electricity" totals

In [ ]:
year = 2020
egrid_plant = validation.load_egrid_plant_file(year)
egrid_plant

In [ ]:
egrid_plant.columns

# Create Emission factors for EIA-930 regions

In [ ]:
years_to_load = [2019, 2020, 2021]

# get a list of all balancing areas for which there is data in EIA-930
ba_reference = load_data.load_ba_reference()
bas_in_eia930 = ba_reference[
    (~ba_reference["timezone_reporting_eia930"].isna())
    & (ba_reference["us_ba"] == "Yes")
    & (~(ba_reference["ba_category"] == "miscellaneous"))
    & (~(ba_reference["retirement_date"].dt.year < min(years_to_load)))
]
ba_list = list(bas_in_eia930.ba_code.unique())


local_tz_to_use = "timezone_local"
datetime_to_use = "datetime_utc"

all_data = []
for ba in ba_list:
    print(ba)
    # load and concat multiple years of data
    ba_data = []
    for year in years_to_load:

        plant_attributes = pd.read_csv(
            outputs_folder(f"{year}/plant_static_attributes_{year}.csv"),
            dtype=get_dtypes(),
        )

        # start by loading data for a single BA
        try:
            ba_year_data = pd.read_csv(
                results_folder(f"{year}/plant_data/hourly/us_units/{ba}.csv")
            )
            print(f"  {year}")

            # make sure the data frame is not empty
            if len(ba_year_data) > 0:
                # assign a fuel category to each plant
                ba_year_data = ba_year_data.merge(
                    plant_attributes[["plant_id_eia", "fuel_category_eia930"]],
                    how="left",
                    on="plant_id_eia",
                    validate="m:1",
                )

                # check that there are no missing fuel categories
                if len(ba_year_data[ba_year_data["fuel_category_eia930"].isna()]) > 0:
                    print(f"Warning: there are missing fuel categories in {ba}")

                # aggregate by fuel category
                ba_year_data = (
                    ba_year_data.groupby(
                        ["fuel_category_eia930", "datetime_utc"], dropna=False
                    )
                    .sum()
                    .reset_index()
                )

                # create columns for adjusted emissions for all pollutants
                # biomass adjustment does not affect these pollutants, only co2
                for pol in ["ch4", "n2o", "nox", "so2"]:
                    ba_year_data[
                        f"{pol}_mass_lb_for_electricity_adjusted"
                    ] = ba_year_data[f"{pol}_mass_lb_for_electricity"]

                # calculate co2eq emissions
                ba_year_data = emissions.calculate_co2e_mass(
                    ba_year_data,
                    year,
                    gwp_horizon=100,
                    ar5_climate_carbon_feedback=True,
                )

                ba_data.append(ba_year_data)
        except FileNotFoundError:
            pass

    try:
        # concat data for all years together
        ba_data = pd.concat(ba_data, axis=0)

        # groupby to make sure we don't have duplicate timestamps
        ba_data = (
            ba_data.groupby(["fuel_category_eia930", "datetime_utc"], dropna=False)
            .sum()
            .reset_index()
        )

        # convert the datetime column to a datetime dtype
        ba_data["datetime_utc"] = pd.to_datetime(ba_data["datetime_utc"])

        # load the local timezone to which each ba reports data to EIA-930
        ba_local_tz = bas_in_eia930.loc[
            ba_reference["ba_code"] == ba, local_tz_to_use
        ].values[0]

        # create a local datetime column
        ba_data["datetime_local"] = ba_data["datetime_utc"].dt.tz_convert(ba_local_tz)

        # create columns for month and hour
        ba_data["year"] = ba_data[datetime_to_use].dt.year
        ba_data["month"] = ba_data[datetime_to_use].dt.month
        ba_data["hour"] = ba_data[datetime_to_use].dt.hour

        # groupby month-hour
        ba_data = (
            ba_data.groupby(["fuel_category_eia930", "year", "month", "hour"])
            .sum()
            .reset_index()
        )

        # calculate emission factors
        for pol in ["co2", "co2e", "nox", "so2"]:
            for pol_type in ["for_electricity", "for_electricity_adjusted"]:
                ba_data[f"{pol}_lb_per_mwh_{pol_type}"] = (
                    ba_data[f"{pol}_mass_lb_{pol_type}"] / ba_data["net_generation_mwh"]
                )

        # add a column for ba code
        ba_data["ba_code"] = ba

        # only keep relevant columns
        key_columns = [
            "ba_code",
            "fuel_category_eia930",
            "year",
            "month",
            "hour",
        ]

        factor_columns = [
            "co2_lb_per_mwh_for_electricity",
            "co2_lb_per_mwh_for_electricity_adjusted",
            "co2e_lb_per_mwh_for_electricity",
            "co2e_lb_per_mwh_for_electricity_adjusted",
            "nox_lb_per_mwh_for_electricity",
            "so2_lb_per_mwh_for_electricity",
        ]

        ba_data = ba_data[key_columns + factor_columns]

        # set all emissions for zero carbon fuels to zero
        zero_carbon_fuels = ["solar", "wind", "hydro", "nuclear"]
        ba_data.loc[
            ba_data["fuel_category_eia930"].isin(zero_carbon_fuels), factor_columns
        ] = 0

        all_data.append(ba_data)
    # let us know if there was not any data to concatenate
    except ValueError:
        print(f"No data available for {ba}")
        pass

all_data = pd.concat(all_data, axis=0)

# only keep data that are in the data years we want
all_data = all_data[all_data["year"].isin(years_to_load)]


In [ ]:
all_data = all_data[all_data["year"].isin(years_to_load)]

In [108]:
all_data.to_csv("../../../../time_varying_emission_factors_for_eia_regions.csv", index=False)

# Create Emission Factors for ISO Regions
 - Need to create a table that maps fuel categories to ISO categories (updating in energy_source_groups)
 - Need to look at aggregate/special categories